In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_df = pd.read_csv('../input/english-handwritten-characters-dataset/english.csv')
data_df.head()

In [ ]:
import torch, cv2
from glob import glob
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class dataset(Dataset) :
    def __init__(self, image_list, image_names, image_labels, image_label_dict, transform, device) :
        self.image_list = image_list
        self.image_names = image_names
        self.image_labels = image_labels
        self.image_label_dict = image_label_dict
        self.transform = transform
        self.device = device
            
    def __len__(self) :
        return len(self.image_list)
    
    def __getitem__(self, index) :
        x = cv2.imread(self.image_list[index])
        x = self.transform(x).to(device)
        
        image_name = image_list[index][48:]
        y = self.image_labels[np.where(self.image_names == image_name)]
        y = self.image_label_dict[y[0]]
        y = torch.LongTensor([y]).to(device)
        
        return x, y
    
image_list = glob('../input/english-handwritten-characters-dataset/Img/*.png')
image_names = data_df["image"].values
image_labels = data_df["label"].values
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Resize((224, 224)) #900, 1200 => 224, 224
])

image_label_dict = dict()
tmp = 0
for label in set(list(image_labels)) :
    image_label_dict[label] = tmp
    tmp += 1
    
train_data = image_list[:-10]
test_data = image_list[-10:]

train_data = dataset(train_data, image_names, image_labels, image_label_dict, transform, device)
train_data = DataLoader(train_data, batch_size = 64, shuffle = True)

test_data = dataset(test_data, image_names, image_labels, image_label_dict, transform, device)
test_data = DataLoader(test_data, batch_size = 1, shuffle = False)

In [ ]:
import torch.nn as nn
from torch.optim import Adam
from torchvision.models import resnet34

class resnet(nn.Module) :
    def __init__(self, output) :
        super().__init__()
        self.model = resnet34(pretrained = False)
        self.model.fc = nn.Linear(512, output)
        
    def forward(self, x) :
        output = self.model(x)
        return output
    
model = resnet(62).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = Adam(model.parameters(), lr = 0.01)

In [ ]:
from tqdm.notebook import tqdm

epoch_num = 30
torch.cuda.empty_cache()

for epoch in range(epoch_num) :
    epoch_loss = 0
    epoch_acc = 0
    for i, (x, y) in tqdm(enumerate(train_data)) :
        y = y.reshape(-1)
        
        predict = model(x)
        loss = criterion(predict, y)
        
        epoch_loss += loss / len(train_data)
        correct_prediction = torch.argmax(predict, 1) == y
        correct_prediction = correct_prediction.sum()
        epoch_acc += correct_prediction
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    epoch_acc = epoch_acc / (64 * len(train_data))
    print('Epoch : {}/{},   loss : {:.5f},    acc : {:.5f}'.format(epoch+1, epoch_num, epoch_loss, epoch_acc))
    
    if epoch_acc > 0.98 : break

In [ ]:
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad() :
    for i, (x, y) in tqdm(enumerate(test_data)) :
        y = y.reshape(-1)
        predict = model(x)
        
        predict = torch.argmax(predict)
        
        chk1, chk2 = False, False
        for key, value in image_label_dict.items() :
            if predict == value :
                predict = key
                chk1 = True
            if y == value :
                y = key
                chk2 = True
                
            if chk1 and chk2 : break
                
        x = x.reshape((3, 224, 224)).to('cpu').numpy()
        plt.imshow(x.transpose(1, 2, 0))
        plt.title(predict)
        plt.show()